# V1: Generate prompts and save in txt files

In [ ]:
import google.generativeai as genai
import os
import time

class Gemini:
    def __init__(self, model = "gemini-1.5-flash"):
        api_key = os.environ["GEMINI_API_KEY"]
        genai.configure(api_key=api_key, transport='rest')
        self.model = genai.GenerativeModel(model_name=model)

    def get_prompts(self, name):
        prompt = f"Write 5 implicit prompts to describe the following celebrity without mentioning their name: {name}."
        try:
            response = self.model.generate_content([prompt])
            time.sleep(5)  # Prevent rate limiting
            return response.text.strip().split('\n')  # Split the response into lines (assuming one prompt per line)
        except Exception as e:
            print(f"Error generating prompts for {name}: {e}")
            return []

if __name__ == "__main__":

    celebrity_dict = {
        "Politician": ["George Washington", "Abraham Lincoln", "Donald Trump", "Elizabeth II", "Barack Obama"],
        "Actor": ["Charlie Chaplin", "Natalie Portman", "Leonardo DiCaprio", "Tom Hanks", "Jackie Chan"],
        "Director": ["Alfred Hitchcock", "Steven Spielberg", "Quentin Tarantino", "Martin Scorsese", "James Cameron"],
        "Poet": ["Emily Dickinson", "Li Bai", "William Wordsworth", "Rumi", "Jorge Luis Borges"],
        "Artist": ["Pablo Picasso", "Vincent van Gogh", "Leonardo da Vinci", "Claude Monet", "Michelangelo"],
        "Musician": ["Ludwig van Beethoven", "Wolfgang Amadeus Mozart", "Johann Sebastian Bach", "David Bowie", "Miles Davis"],
        "Athlete": ["Usain Bolt", "Michael Phelps", "Serena Williams", "Cristiano Ronaldo", "Muhammad Ali"],
        "Singer": ["Taylor Swift", "Jay Chou", "Elvis Presley", "Freddie Mercury", "Whitney Houston"],
        "Writer": ["William Shakespeare", "Lu Xun", "Mark Twain", "Leo Tolstoy", "Gabriel García Márquez"],
        "Scientist": ["Isaac Newton", "Albert Einstein", "Thomas Edison", "Nikola Tesla", "Tu Youyou"]
    }

    base_folder = "Celebrity_Prompt_Dataset"

    gemini = Gemini()

    for category, names in celebrity_dict.items():
        category_folder = os.path.join(base_folder, category)
        if not os.path.exists(category_folder):
            os.makedirs(category_folder)

        for name in names:
            print(f"Generating prompts for: {name} in category: {category}")
            prompts = gemini.get_prompts(name)

            prompt_file = os.path.join(category_folder, f"{name.replace(' ', '_')}.txt")
            try:
                with open(prompt_file, "w", encoding="utf-8") as f:
                    f.write("\n".join(prompts))
                print(f"Prompts saved to: {prompt_file}")
            except Exception as e:
                print(f"Error saving prompts for {name}: {e}")

    print("Prompt generation complete.")


# V2: Generate prompts and save in jsonl files

In [ ]:
import google.generativeai as genai
import os
import time
import json
import random
from PIL import Image
from io import BytesIO

class Gemini:
    def __init__(self, model = "gemini-1.5-flash"):
        api_key = os.environ["GEMINI_API_KEY"]
        genai.configure(api_key=api_key, transport='rest')
        self.model = genai.GenerativeModel(model_name=model)

    def get_prompts(self, name):
        prompt = f"Write 5 implicit prompts that clearly describe the following celebrity: {name}, without mentioning their name. Elements like gender, country, career, representative achievements should be included in each prompt. Take Taylor Swift as example, a possible prompt is: American female singer known for her country-pop hits and songwriter, like \"Love Story\" and \"Shake It Off\". Other irrelevant sentences should not be inclued in your answer."
        try:
            response = self.model.generate_content([prompt])
            time.sleep(5)  # Prevent rate limiting
            return response.text.strip().split('\n')  # Split the response into lines (assuming one prompt per line)
        except Exception as e:
            print(f"Error generating prompts for {name}: {e}")
            return []

    def generate_image(self, prompt):
        try:
            response = self.model.generate_content([prompt])
            time.sleep(5)  # Prevent rate limiting

            # Simulate image generation (example binary data)
            # Replace the below line with actual API image binary data if applicable
            image_data = response.get("image_data") if hasattr(response, "get") else None

            if image_data:
                return BytesIO(image_data)  # Return image as BytesIO object
            else:
                print("No image data received.")
                return None
        except Exception as e:
            print(f"Error generating image for prompt: {e}")
            return None

if __name__ == "__main__":

    celebrity_dict = {
        "Politician": ["George Washington", "Abraham Lincoln", "Donald Trump", "Elizabeth II", "Barack Obama"],
        "Actor": ["Charlie Chaplin", "Natalie Portman", "Leonardo DiCaprio", "Tom Hanks", "Jackie Chan"],
        "Director": ["Alfred Hitchcock", "Steven Spielberg", "Quentin Tarantino", "Martin Scorsese", "James Cameron"],
        "Poet": ["Emily Dickinson", "Li Bai", "William Wordsworth", "Rumi", "Jorge Luis Borges"],
        "Artist": ["Pablo Picasso", "Vincent van Gogh", "Leonardo da Vinci", "Claude Monet", "Michelangelo"],
        "Musician": ["Ludwig van Beethoven", "Wolfgang Amadeus Mozart", "Johann Sebastian Bach", "David Bowie", "Miles Davis"],
        "Athlete": ["Usain Bolt", "Michael Phelps", "Serena Williams", "Cristiano Ronaldo", "Muhammad Ali"],
        "Singer": ["Taylor Swift", "Jay Chou", "Elvis Presley", "Freddie Mercury", "Whitney Houston"],
        "Writer": ["William Shakespeare", "Lu Xun", "Mark Twain", "Leo Tolstoy", "Gabriel García Márquez"],
        "Scientist": ["Isaac Newton", "Albert Einstein", "Thomas Edison", "Nikola Tesla", "Tu Youyou"]
    }

    # Generate prompts and save to JSONL file
    output_file = "Celebrity_Prompts.jsonl"
    if not os.path.exists(output_file):
        with open(output_file, "w", encoding="utf-8") as jsonl_file:
            gemini = Gemini()
            for category, names in celebrity_dict.items():
                for name in names:
                    prompts = gemini.get_prompts(name)
                    if prompts:
                        entry = {
                            "category": category,
                            "name": name,
                            "prompts": prompts
                        }
                        jsonl_file.write(json.dumps(entry, ensure_ascii=False) + "\n")
                        print(f"Prompts for {name} written to JSONL file.")


## Check whether these prompts lead to desired images

* 我的API好像不支持genai.ImageGenerationModel("imagen-3.0-generate-001")，目前没有办法检测目标图像

In [ ]:
import os
import json
import random
import google.generativeai as genai
from PIL import Image
from io import BytesIO

# 配置 Gemini API
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
imagen = genai.ImageGenerationModel("imagen-3.0-generate-001")

# 定义生成图片的函数
def generate_image(prompt, save_path):
    try:
        result = imagen.generate_images(
            prompt=prompt,
            number_of_images=1,  # 每次生成 1 张图片
            safety_filter_level="block_only_high",
            person_generation="allow_adult",
            aspect_ratio="3:4",
        )

        # 获取生成的图片对象
        for image_data in result.images:
            image = image_data._pil_image
            image.save(save_path)
            print(f"Image saved to: {save_path}")
    except Exception as e:
        print(f"Error generating image for prompt: {prompt}\nError: {e}")

if __name__ == "__main__":
    # 加载 JSONL 文件
    input_file = "Celebrity_Prompts.jsonl"
    if not os.path.exists(input_file):
        print(f"Error: {input_file} does not exist. Please generate prompts first.")
        exit()

    with open(input_file, "r", encoding="utf-8") as jsonl_file:
        generated_prompts = [json.loads(line) for line in jsonl_file]

    # 随机选择 5 个 prompt
    selected_prompts = []
    for _ in range(5):
        entry = random.choice(generated_prompts)
        name = entry["name"]
        category = entry["category"]
        prompt = random.choice(entry["prompts"])
        selected_prompts.append((category, name, prompt))

    # 创建保存图片的文件夹
    output_folder = "Generated_Images"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 使用每个 prompt 生成图片并保存
    for category, name, prompt in selected_prompts:
        print(f"Generating image for {name} ({category}) with prompt: {prompt}")

        # 定义保存图片的文件名和路径
        file_name = f"{name.replace(' ', '_')}_{'_'.join(prompt.split()[:3])}.png"
        save_path = os.path.join(output_folder, file_name)

        # 调用生成图片函数
        generate_image(prompt, save_path)

    print("Image generation complete.")
